# **LAB 01-Feed Forward & Back-Propagation Learning Algorithm**

### **20th January 2025**
# **REDDIPALLI SAI CHARISH**
# **CS22B1095**


---



---



---







#**QUESTION-1**
Implement the Perceptron algorithm from scratch in Python.
*   Initialize the weights with [0 0 0] and a learning rate of 0.0001.
*   For each iteration, calculate the output of the Perceptron for each input in the training set.
*  Use MSE to computer the error for all samples
* Update the weights using the gradient descent procedure.
*   Repeat the above steps until the Perceptron converges or a maximum number of iterations is reached.
* Test the trained Perceptron on a separate test set, explain how you came up with the test set.
* Use the step function as an  activation function in the output layer
Use the IRIS Dataset for the above, considering all four features: sepal length, sepal width, petal length, and petal width, but only two classes -  Setosa, and Versicolor.  Drop the feature vectors of the other class.


In [127]:
import numpy as np
import pandas as pd

## **Perceptron**

In [128]:
class Perceptron:
  def __init__(self,lr=0.0001, epochs=100):
    self.lr = lr
    self.epochs = epochs
    self.acv_func = self.step_func
    self.w = None

  def step_func(self, x):
    return np.where(x >= 0, 1, 0)

  def fit(self,X,y):
    X=np.insert(X,0,1,axis=1)
    n_samples,n_features = X.shape
    self.w = np.zeros(n_features)

    while self.epochs > 0:
      flag = 0
      for i,x_i in enumerate(X):
        dot_output = np.dot(x_i,self.w)
        y_pred = self.acv_func(dot_output)
        if y_pred != y[i]:
          self.w += self.lr * (y[i] - y_pred) * x_i
          flag = 1
      if flag == 0:
        break
      self.epochs -= 1

  def predict(self,X):
    X=np.insert(X,0,1,axis=1)
    return self.acv_func(np.dot(X,self.w))


In [129]:

iris_df = pd.read_csv("iris.csv").drop("Id", axis=1)
iris_df = iris_df[iris_df['Species'].isin(['Iris-setosa', 'Iris-versicolor'])]
print(f"Count: {len(iris_df)}")
iris_df

Count: 100


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,Iris-versicolor
96,5.7,2.9,4.2,1.3,Iris-versicolor
97,6.2,2.9,4.3,1.3,Iris-versicolor
98,5.1,2.5,3.0,1.1,Iris-versicolor


In [130]:

X = iris_df.drop("Species", axis=1).values
y = iris_df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1}).values

In [131]:

def train_test_split(X, y, test_size=0.9):
    n_classes = len(np.unique(y))
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(n_classes):
        X_class = X[y == i]
        y_class = y[y == i]
        n_samples = len(X_class)
        n_train = int(n_samples * (1 - test_size))
        X_train.extend(X_class[:n_train])
        X_test.extend(X_class[n_train:])
        y_train.extend(y_class[:n_train])
        y_test.extend(y_class[n_train:])
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Test dataset for each class:")
print(f"{np.asarray(np.unique(y_test, return_counts=True)).T}")



Test dataset for each class:
[[ 0 10]
 [ 1 10]]


In [132]:
perceptron=Perceptron(lr=0.0001,epochs=100)
perceptron.fit(X_train,y_train)

In [133]:
y_pred=perceptron.predict(X_test)
result=pd.DataFrame({'Actual':y_test,'Predicted':y_pred})
result


,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [134]:
accuracy=np.mean(y_pred==y_test)*100
accuracy

100.0

In [135]:
W1 = np.array([[0.5, 1.5, 0.8], [0.8, 0.2, -1.6]])
W2 = np.array([[0.9, -1.7, 1.6], [1.2, 2.1, -0.2]])

In [136]:
X1 = np.array([1.0, 0.7, 1.2])
X2 = np.array([1.0, 0.0, 0.0])
t = np.array([1.0, 0.0])

In [137]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [138]:
def sigmoid_output(y: np.array):
    return np.array([1 if i > 0.5 else 0 for i in y])

In [139]:
def feed_forward(feature_vector, weight_vector):
    return sigmoid(np.dot(weight_vector, feature_vector))

In [140]:
def back_propagation(W, X, delta_vector, learning_rate):
    n, m = W.shape
    for j in range(n):
        for i in range(m):
            W[j, i] = W[j, i] - learning_rate * delta_vector[j] * X[i]
    return W

In [141]:
def delta_hidden_layer(next_delta_vector, W, output_vector):
    n, m = W.shape
    delta_vector = np.zeros(m - 1)

    for i in range(m - 1):
        for j in range(n):
            delta_vector[j] += next_delta_vector[i] * W[i , j + 1]

    delta_vector = output_vector * (1 - output_vector) * delta_vector
    return delta_vector

In [142]:
def mlp(X1, X2, W1, W2, t, learning_rate = 0.001, max_epochs = 1000):
     for epoch in range(max_epochs):
          print(f"Epoch {epoch + 1}")
          o1 = feed_forward(X1, W1)
          X2[1], X2[2] = o1[0], o1[1]
          o2 = feed_forward(X2, W2)
          output = sigmoid_output(o2)
          if(np.array_equal(output, t)):
               print(f"Converged after {epoch + 1} epochs")
               break
          else:
               delta_2_vector = (o2 - t) * o2 * (1 - o2)
               # print(f"Delta 2: {delta_2_vector}")
               W2 = back_propagation(W2, X2, delta_2_vector, learning_rate)
               # print("O1 : ", o1)
               delta_1_vector = delta_hidden_layer(delta_2_vector, W2, o1)
               # print(f"Delta 1: {delta_1_vector}")
               W1 = back_propagation(W1, X1, delta_1_vector, learning_rate)
          print(W1, '\n\n' , W2)
     else:
          print("Did not converge after max epochs")
     return W1, W2

In [143]:
W1, W2 = mlp(X1, X2, W1, W2, t, 0.5, 1000)
print("")
print("")

print("Final weight vector:")
print("W1:", W1)
print("W2:", W2)

Epoch 1
[[ 0.48928025  1.49249617  0.7871363 ]
 [ 0.8229341   0.21605387 -1.57247908]] 

 [[ 0.96886855 -1.63630762  1.61879366]
 [ 1.18005027  2.08154969 -0.20544412]]
Epoch 2
[[ 0.47892982  1.48525087  0.77471578]
 [ 0.84552115  0.2318648  -1.54537462]] 

 [[ 1.03368231 -1.57650867  1.63735805]
 [ 1.15929163  2.0623972  -0.21138994]]
Epoch 3
[[ 0.46899918  1.47829943  0.76279902]
 [ 0.86741089  0.24718763 -1.51910693]] 

 [[ 1.09378691 -1.52118633  1.65539794]
 [ 1.13766631  2.04249252 -0.2178806 ]]
Epoch 4
[[ 0.45949453  1.47164617  0.75139344]
 [ 0.88835515  0.2618486  -1.49397382]] 

 [[ 1.14890458 -1.47057303  1.672693  ]
 [ 1.11511214  2.02178153 -0.22495775]]
Epoch 5
[[ 0.45038709  1.46527096  0.7404645 ]
 [ 0.90821357  0.2757495  -1.47014372]] 

 [[ 1.19907099 -1.42461242  1.68910477]
 [ 1.09156163  2.00020543 -0.23266222]]
Epoch 6
[[ 0.4416251   1.45913757  0.72995012]
 [ 0.92693896  0.28885727 -1.44767325]] 

 [[ 1.24453851 -1.383051    1.7045674 ]
 [ 1.06694093  1.97769988 